Import Dependencies

In [3]:
import pandas as pd
import numpy as np

## Load Dataset

In [8]:
df_cose = pd.read_csv("./data/cos-e/parsed-cose-train.csv")
df_cose

,id,question,choice_0,choice_1,choice_2,choice_3,choice_4,label,human_expl_open-ended
0,075e483d21c29a511267ef62bedc0461,The sanctions against the school were a punish...,ignore,enforce,authoritarian,yell at,avoid,0,Not sure what else could be a common ground
1,61fe6e879ff18686d7552425a36344c8,Sammy wanted to go to where the people were. ...,race track,populated areas,the desert,apartment,roadblock,1,People will be in populated areas.
2,02e821a3e53cb320790950aab4489e85,Google Maps and other highway and street GPS s...,united states,mexico,countryside,atlas,oceans,3,atlases were collections of highway and street...
3,23505889b94e880c3e89cff4ba119860,"The fox walked from the city into the forest, ...",pretty flowers.,hen house,natural habitat,storybook,dense forest,2,Usually the habitat of a fox is forest and it ...
4,e8a8b3a2061aa0e6d7c6b522e9612824,What home entertainment equipment requires cable?,radio shack,substation,cabinet,television,desk,3,television is the only option that is a home e...
...,...,...,...,...,...,...,...,...,...
7186,28ab300ef821e57e19be3f757842dd62,Where might I find seating while waiting to ta...,rest area,bus depot,theatre,bus stop,church,1,I FIND SEATING TO WAIT IN BUS DEPOT FOR PUBLIC...
7187,f1eb6055fa8a1ec94fa6d710d9a6741b,Something that you need to have inside of you ...,workers,money,determination,funding,creativity,2,opening a business is tough so you need to hav...
7188,f1b2a30a1facff543e055231c5f90dd0,What would someone need to do if he or she wan...,consequences,being ridiculed,more money,more funding,telling all,4,To go public is to be revealing information an...
7189,22d0eea15e10be56024fd00bb0e4f72f,Where would you buy jeans in a place with a la...,shopping mall,laundromat,hospital,clothing store,thrift store,0,When submitting your products to Merchant Cent...


## Get Explanations

### GPT explanation

In [16]:
# imports
import os
import ast  # for converting embeddings saved as strings back to arrays
import openai  
import pandas as pd
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search
openai.api_key = os.environ["OPENAI_API_KEY"]

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
# GPT_MODEL = "gpt-3.5-turbo"
GPT_MODEL = "gpt-4" # According to OpenAI, GPT-4 is more responsive to system messages, whereas 3.5 would rely on more of the user input.


In [17]:
# an example nle question
premise = "Which of the following should you not bring a fox in"
choices = ['hen house', 'England', 'elementary school']
query = f"""Question: {premise}:
Choices: {"  ".join([f"{i+1}.{choices[i]}" for i in range(len(choices))])}"""

sys_msg = """
Make a selection and explain your reasoning.
Reply in the following format: the first line is contains only the choice number,
and the second line is the explanation.
"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': sys_msg},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

model_response = response['choices'][0]['message']['content']

In [18]:
y_pred, model_nle = model_response.split('\n')
y_pred = int(y_pred)
y_pred, model_nle 

(1,
 'Bringing a fox into a hen house would be disastrous, as foxes are natural predators of chickens and would likely cause harm or death to the hens.')

## Obtain NLE Embeddings

In [ ]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large', verbose=0)
roberta.eval()  # disable dropout (or leave in train mode to finetune)
roberta.to('cuda')

In [18]:
to_tok = list(df_cose.columns)
for x in ['id', 'label']:
    to_tok.remove(x)
tok_df_cose = df_cose.copy()
tok_df_cose[to_tok] = tok_df_cose[to_tok].apply(lambda x: [roberta.encode(str(s)) for s in x])
tok_df_cose

['question',
 'choice_0',
 'choice_1',
 'choice_2',
 'choice_3',
 'choice_4',
 'human_expl_open-ended']

In [ ]:

tok_model_nle = roberta.encode(model_nle)
tok_choices = [roberta.encode(ch) for ch in choices]

# embed the tokens using the last layer feature of the model
emb_model_nle = roberta.extract_features(tok_model_nle)
tok_choices = [roberta.extract_features(ch) for ch in tok_choices]


## Calculate SHAP Score

In [ ]:
import shap
X100 = shap.utils.sample(X, 100)
shap.partial_dependence_plot(
    your_feature_name, your_function_to_get_y_pred, X100, ice=False,
    model_expected_value=True, feature_expected_value=True
)

In [ ]:
explainer = shap.Explainer(your_function_to_get_y_pred, X100)
shap_values = explainer(X)